# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2 toc-item"><a href="#Generate-List-of-Data" data-toc-modified-id="Generate-List-of-Data-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev1 toc-item"><a href="#Modules-vs-Subgraphs" data-toc-modified-id="Modules-vs-Subgraphs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modules vs Subgraphs</a></div><div class="lev2 toc-item"><a href="#Module-Sensitivity:-Inter/Intra-Module-Connectivity-of-Subgraphs" data-toc-modified-id="Module-Sensitivity:-Inter/Intra-Module-Connectivity-of-Subgraphs-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Module-Sensitivity: Inter/Intra Module Connectivity of Subgraphs</a></div><div class="lev3 toc-item"><a href="#Plot-Time-Averaged-Module-Sensitivity-Index" data-toc-modified-id="Plot-Time-Averaged-Module-Sensitivity-Index-211"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Plot Time-Averaged Module Sensitivity Index</a></div><div class="lev3 toc-item"><a href="#Relate-Module-Reconfiguration-to-Subgraph-Dynamics" data-toc-modified-id="Relate-Module-Reconfiguration-to-Subgraph-Dynamics-212"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Relate Module-Reconfiguration to Subgraph Dynamics</a></div><div class="lev2 toc-item"><a href="#Temporal-Expression-of-Module-Sensitive-Subgraphs" data-toc-modified-id="Temporal-Expression-of-Module-Sensitive-Subgraphs-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Temporal Expression of Module-Sensitive Subgraphs</a></div><div class="lev3 toc-item"><a href="#Plot-Correlation-between-MSI-and-Subgraph-Expression" data-toc-modified-id="Plot-Correlation-between-MSI-and-Subgraph-Expression-221"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Plot Correlation between MSI and Subgraph Expression</a></div><div class="lev3 toc-item"><a href="#Plot-temporal-expression" data-toc-modified-id="Plot-temporal-expression-222"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Plot temporal expression</a></div>

# Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)

path_CoreData = '/Users/akhambhati/Remotes/CORE.fMRI_multiband.mmattar/restdata'
path_PeriphData = '/Users/akhambhati/Remotes/RSRCH.NMF_Subnetworks'
path_ModData = path_PeriphData + '/e02b-DynFuncModule-Population'
path_SubData = path_PeriphData + '/e03b-DynFuncSubgraph-Population'
path_ExpData = path_PeriphData + '/e04b-CmpModuleSubgraph-Population'

for path in [path_CoreData, path_PeriphData, path_ModData, path_SubData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)     
        
if not os.path.exists('./e04b-Figures'):
    os.makedirs('./e04b-Figures')

## Generate List of Data

In [ ]:
data_dict = {'module_path': glob.glob('{}/Module_Optimization.ModAssign.*.mat'.format(path_ModData)),
             'subgraph_path': '{}/NMF_Optimization.consensus_subgraph.npz'.format(path_SubData),
             'cfg_path': '{}/NMF_Optimization.CfgMatr.npz'.format(path_SubData),
             'subgraph_geom_null_path': glob.glob('{}/NMF_GeomNull.subgraph_seed-*.npz'.format(path_SubData)),
             'subgraph_edge_null_path': glob.glob('{}/NMF_EdgeNull.subgraph_seed-*.npz'.format(path_SubData))}

# Modules vs Subgraphs

## Module-Sensitivity: Inter/Intra Module Connectivity of Subgraphs

In [ ]:
def map_module_to_subgraph(module_assign, adj_subgraph):
    """
    Map module onto subgraph and determine inter/intra module connectivity
    
    Parameters
    ----------
        module_assign: ndarray, shape (N,)
            Assignment of N nodes to modules in a single time window
            Each module should have an integer identifier
        
        adj_subgraph: ndarray, shape (N, N)
            Subgraph adjacency matrix
            
    Returns
    -------
        msi: float, -1 <= x <= 1
            The module-sensitivity index where msi < 0 implies 
            greater connectivity between modules and msi > 0 implies
            greater connectivity within modules
    """
    
    # Get data params
    n_node = len(module_assign)
    assert (n_node, n_node) == adj_subgraph.shape
    
    # Get the module identifiers
    module_id = np.unique(module_assign)
        
    # Compute inter_module_conn
    if len(module_id) == 1: 
        inter_module_conn = 0
    else:      
        inter_module_conn = []
        for m_id in module_id:
            in_nodes = np.flatnonzero(module_assign == m_id)
            out_nodes = np.setdiff1d(np.arange(n_node), in_nodes)
            
            in_1, out_2 = np.meshgrid(in_nodes, out_nodes)                
            inter_module_conn.append(np.mean(subgraph_adj[in_1, out_2]))
        inter_module_conn = np.nanmean(inter_module_conn)

    # Compute intra_module_conn
    intra_module_conn = []
    for m_id in module_id:
        in_nodes = np.flatnonzero(module_assign == m_id)

        in_1, in_2 = np.meshgrid(in_nodes, in_nodes)
        intra_module_conn.append(np.mean(subgraph_adj[in_1, in_2]))
    intra_module_conn = np.nanmean(intra_module_conn)   
    
    msi = (intra_module_conn - inter_module_conn) / \
          (intra_module_conn + inter_module_conn)

    return msi

In [ ]:
# Load cfg data
data_cfg = np.load(data_dict['cfg_path'], mmap_mode='r')
subj_coef_lbl = np.array([name.split('.')[0] for name in data_cfg['cfg_name']])
subj_lbl = np.unique(subj_coef_lbl)
n_subj = len(subj_lbl)
n_win = np.int(np.ceil(len(subj_coef_lbl) / n_subj))

# Load Subgraph Data
data_sub = np.load(data_dict['subgraph_path'], mmap_mode='r')
fac_subgraph = data_sub['fac_subnet'] / data_sub['fac_subnet'].max()

# Get Data Params
n_fac = fac_subgraph.shape[0]
n_conn = fac_subgraph.shape[1]
n_node = np.int(np.ceil(np.sqrt(n_conn*2)))
n_mod_seed = len(data_dict['module_path'])
n_null = len(data_dict['subgraph_geom_null_path'])

# Initialize buckets
msi_real = np.nan*np.zeros((n_fac, n_subj, n_win, n_mod_seed))
msi_geom_null = np.nan*np.zeros((n_fac, n_subj, n_win, n_mod_seed, n_null))
msi_geom_null = np.nan*np.zeros((n_fac, n_subj, n_win, n_mod_seed, n_null))

# Real Intra/Inter Module Subgraph Connectivity
for f_id in xrange(n_fac):
    subgraph_adj = convert_conn_vec_to_adj_matr(fac_subgraph[f_id, :])

    for mseed_id, mseed_path in enumerate(data_dict['module_path']):
        data_mseed = io.loadmat(mseed_path)
        
        for s_id in xrange(n_subj):
            
            module_assign = np.array(data_mseed['Ssubj'][0, s_id], float)
            n_node, n_win = module_assign.shape            
            for w_id in xrange(n_win):
                msi = map_module_to_subgraph(module_assign[:, w_id],
                                             subgraph_adj)
                msi_real[f_id, s_id, w_id, mseed_id] = msi
        break

                
# Geometric Null Intra/Inter Module Subgraph Connectivity
for n_i, null_path in enumerate(data_dict['subgraph_geom_null_path']):
    data_null_sub = np.load(null_path, mmap_mode='r')
    fac_subgraph = data_null_sub['fac_subnet'] / data_null_sub['fac_subnet'].max()
    
    for f_id in xrange(n_fac):
        subgraph_adj = convert_conn_vec_to_adj_matr(fac_subgraph[f_id, :])

        for mseed_id, mseed_path in enumerate(data_dict['module_path']):
            data_mseed = io.loadmat(mseed_path)

            for s_id in xrange(n_subj):

                module_assign = np.array(data_mseed['Ssubj'][0, s_id], float)
                n_node, n_win = module_assign.shape            
                for w_id in xrange(n_win):
                    msi = map_module_to_subgraph(module_assign[:, w_id],
                                                 subgraph_adj)
                    msi_geom_null[f_id, s_id, w_id, mseed_id, n_i] = msi    
                    
                    
# Edge Null Intra/Inter Module Subgraph Connectivity
for n_i, null_path in enumerate(data_dict['subgraph_edge_null_path']):
    data_null_sub = np.load(null_path, mmap_mode='r')
    fac_subgraph = data_null_sub['fac_subnet'] / data_null_sub['fac_subnet'].max()
    
    for f_id in xrange(n_fac):
        subgraph_adj = convert_conn_vec_to_adj_matr(fac_subgraph[f_id, :])

        for mseed_id, mseed_path in enumerate(data_dict['module_path']):
            data_mseed = io.loadmat(mseed_path)

            for s_id in xrange(n_subj):

                module_assign = np.array(data_mseed['Ssubj'][0, s_id], float)
                n_node, n_win = module_assign.shape            
                for w_id in xrange(n_win):
                    msi = map_module_to_subgraph(module_assign[:, w_id],
                                                 subgraph_adj)
                    msi_edge_null[f_id, s_id, w_id, mseed_id, n_i] = msi   

                    
np.savez('{}/Module_Sensitivity.npz'.format(path_ExpData),
         msi_real=msi_real,
         msi_geom_null=msi_geom_null,
         msi_edge_null=msi_edge_null)

### Plot Time-Averaged Module Sensitivity Index 

In [ ]:
df = np.load('{}/Module_Sensitivity_Index.npz'.format(path_ExpData))
msi_pop = df['msi_pop']
msi_pop_geom_null = df['msi_pop_geom_null']
msi_pop_edge_null = df['msi_pop_edge_null']

# Get data params
n_subj, n_null, n_fac, n_win = msi_pop_geom_null.shape

# Sort the topographic sensitivity for subgraphs of each subject
msi_pop_sorted = np.sort(np.nanmean(msi_pop, axis=-1), axis=1)
msi_pop_geom_null_sorted = np.sort(np.nanmean(np.nanmean(msi_pop_geom_null, axis=-1), axis=1))
msi_pop_edge_null_sorted = np.sort(np.nanmean(np.nanmean(msi_pop_edge_null, axis=-1), axis=1))

alpha = 0.05 / n_fac

print('\n\n----- Geometric Null ------')
for ii in xrange(n_fac):
    tval, pval = stats.ttest_rel(msi_pop_sorted[:, ii], msi_pop_geom_null_sorted[:, ii])
    if pval < alpha:
        sig = '*'
    else:
        sig = ' '    
    print('{}- t={:0.5}, p={:0.5}, sig={}'.format(ii+1, tval, pval, sig))


print('\n\n----- Edge Null ------')
for ii in xrange(n_fac):
    tval, pval = stats.ttest_rel(msi_pop_sorted[:, ii], msi_pop_edge_null_sorted[:, ii])
    if pval < alpha:
        sig = '*'
    else:
        sig = ' '    
    print('{}- t={:0.5}, p={:0.5}, sig={}'.format(ii+1, tval, pval, sig))
    
    
# Plot Module Sensitivity
plt.figure()
ax = plt.subplot(111)

ax.plot(np.linspace(1-0.25, n_fac+0.25, n_fac),
        np.percentile(msi_pop_geom_null_sorted, 50, axis=0),
        color='k', linewidth=0.5)
ax.fill_between([1-0.25, n_fac+0.25],
                y1=np.percentile(msi_pop_geom_null_sorted, 5),
                y2=np.percentile(msi_pop_geom_null_sorted, 95),
                color=[1.00, 0.25, 0.25], alpha=0.5, lw=0)


ax.plot(np.linspace(1-0.25, n_fac+0.25, n_fac),
        np.percentile(msi_pop_edge_null_sorted, 50, axis=0),
        color='k', linewidth=0.5)
ax.fill_between([1-0.25, n_fac+0.25],
                y1=np.percentile(msi_pop_edge_null_sorted, 5),
                y2=np.percentile(msi_pop_edge_null_sorted, 95),
                color=[0.75, 0.75, 0.75], alpha=0.5, lw=0)

clr_list = []
for ii in xrange(n_fac):
    clr_list.append([0.0, 0.375, 1.0])
bplot = ax.boxplot(msi_pop_sorted, patch_artist=True);
Echobase.Plotting.fig_format.set_box_color(bplot, 'k', clr_list)

# Axis Settings
ax.set_xlim([0, 17])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Module Sensitivity');
ax.set_xlabel('Subgraphs');

plt.savefig('./e04a-Figures/Module_Sensitivity_Index.svg')
plt.show()
plt.close()              

### Relate Module-Reconfiguration to Subgraph Dynamics

In [ ]:
df = np.load('{}/Module_Sensitivity_Index.npz'.format(path_ExpData))
msi_pop = df['msi_pop']
msi_pop_geom_null = df['msi_pop_geom_null']
msi_pop_edge_null = df['msi_pop_edge_null']

# Get data params
n_subj, n_null, n_fac, n_win = msi_pop_geom_null.shape

# Initialize buckets
mod_flex_pop = np.zeros(n_subj)

subg_flex_pop = np.zeros(n_subj)
subg_flex_pop_geom_null = np.zeros((n_subj, n_null, n_fac))
subg_flex_pop_edge_null = np.zeros((n_subj, n_null, n_fac))

dyn_metr = lambda cf: np.nanmean(np.abs(np.diff(cf)))

for subj_i, subj in enumerate(subj_dict.iterkeys()):
    print(" -- Processing: {}".format(subj))
    
    # Compute module flexibility
    data_mod = np.load(subj_dict[subj]['module_path'], mmap_mode='r')
    module_assign = data_mod['module_assign']
    mod_flex_pop[subj_i] = np.mean(np.mean(np.abs(np.diff(module_assign, axis=0)) > 0, axis=0))

    # Load True Subgraphs
    data_sub = np.load(subj_dict[subj]['subgraph_path'], mmap_mode='r')    
    fac_subgraph = data_sub['fac_subnet'] / data_sub['fac_subnet'].max()
    fac_coef = data_sub['fac_coef'] / data_sub['fac_coef'].max()
    
    msi_high_ix = np.argsort(np.nanmean(msi_pop[subj_i, ...], axis=-1))[-1]
    #subg_flex_pop[subj_i] = np.nanmean(np.abs(np.diff(fac_coef[msi_high_ix, :])))
    subg_flex_pop[subj_i] = dyn_metr(fac_coef[msi_high_ix, :])

    # Subgraph Dynamics for Geometric Null 
    for n_i, null_path in enumerate(subj_dict[subj]['subgraph_geom_null_path']):
        data_null_sub = np.load(null_path, mmap_mode='r')
        fac_coef = data_null_sub['fac_coef'] / data_null_sub['fac_coef'].max()
        
        msi_high_ix = np.argsort(np.nanmean(msi_pop_geom_null[subj_i, n_i, ...], axis=-1))[-1]
        for f_i in xrange(n_fac):
            subg_flex_pop_geom_null[subj_i, n_i, f_i] = dyn_metr(fac_coef[msi_high_ix, :])

    # Subgraph Dynamics for Edge Null 
    for n_i, null_path in enumerate(subj_dict[subj]['subgraph_edge_null_path']):
        data_null_sub = np.load(null_path, mmap_mode='r')
        fac_coef = data_null_sub['fac_coef'] / data_null_sub['fac_coef'].max()
        
        msi_high_ix = np.argsort(np.nanmean(msi_pop_edge_null[subj_i, n_i, ...], axis=-1))[-1]
        for f_i in xrange(n_fac):
            fac_coef_norm = (fac_coef / fac_coef.max())[msi_high_ix, :]
            subg_flex_pop_edge_null[subj_i, n_i, f_i] = dyn_metr(fac_coef[msi_high_ix, :])

            
%matplotlib inline
        
# Module Flexibility vs Temporal Energy
m_flex, b_flex, rho_flex, pv_flex, _ = stats.linregress(mod_flex_pop, subg_flex_pop)

rho_geom_null = []
rho_edge_null = []
rho_prsn_null = []   
for n_i in xrange(n_null):
    for f_i in xrange(n_fac):        
        rho_geom_null.append(stats.pearsonr(mod_flex_pop, subg_flex_pop_geom_null[:, n_i, f_i])[0])
        rho_edge_null.append(stats.pearsonr(mod_flex_pop, subg_flex_pop_edge_null[:, n_i, f_i])[0])
        rho_prsn_null.append(stats.pearsonr(mod_flex_pop, np.random.permutation(subg_flex_pop))[0])
        
pv_geom_null = np.mean(rho_geom_null < rho_flex)
pv_edge_null = np.mean(rho_edge_null < rho_flex)
pv_prsn_null = np.mean(rho_prsn_null < rho_flex)
    
print('r={}, p_prsn={}, p_geom={}, p_edge={}'.format(rho_flex, pv_prsn_null, pv_geom_null, pv_edge_null))

plt.figure(); 
ax = plt.subplot(111);
x = np.array([np.min(mod_flex_pop)*0.95, np.max(mod_flex_pop)*1.05])
ax.scatter(mod_flex_pop, subg_flex_pop, color='k', marker='s', alpha=0.6, s=75.0, lw=0)
ax.plot(x, m_flex*x + b_flex, color='k')

# Axis Settings
#ax.set_xlim([0.06, 0.12])
#ax.set_ylim([0, 0.08])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Subgraph Flexibility');
ax.set_xlabel('Module Flexibility');

#plt.savefig('./e04a-Figures/Module_Flexibility-Subgraph_Energy.svg')
plt.show()
plt.close()              

## Temporal Expression of Module-Sensitive Subgraphs

In [ ]:
df = np.load('{}/Module_Sensitivity_Index.npz'.format(path_ExpData))
msi_pop = df['msi_pop']
msi_pop_geom_null = df['msi_pop_geom_null']
msi_pop_edge_null = df['msi_pop_edge_null']

# Get data params
n_subj, n_null, n_fac, n_win = msi_pop_geom_null.shape

# Compute Real MSI x Expression Coefs
msi_coef = np.zeros((n_subj, n_fac))
for subj_i, subj in enumerate(subj_dict.iterkeys()):
    
    # Load Subject Data
    data_sub = np.load(subj_dict[subj]['subgraph_path'], mmap_mode='r')
    fac_coef = data_sub['fac_coef'] / data_sub['fac_coef'].max()
    
    # Get data params
    n_fac = fac_coef.shape[0]
    for fac_i in xrange(n_fac):
        good_tw = np.flatnonzero(~np.isnan(msi_pop[subj_i, fac_i, :]))
        rho, pv = stats.pearsonr(msi_pop[subj_i, fac_i, good_tw], fac_coef[fac_i, good_tw])
        if np.isnan(rho):
            continue
        msi_coef[subj_i, fac_i] = rho
        
        
# Compute Geometric Null MSI x Expression Coefs
msi_coef_geom_null = np.zeros((n_subj, n_null, n_fac))
for subj_i, subj in enumerate(subj_dict.iterkeys()):
    for n_i, null_path in enumerate(subj_dict[subj]['subgraph_geom_null_path']):
        
        # Load Subject Data
        data_null_sub = np.load(null_path, mmap_mode='r')
        fac_coef = data_null_sub['fac_coef'] / data_null_sub['fac_coef'].max()
    
        # Get data params
        n_fac = fac_coef.shape[0]
        for fac_i in xrange(n_fac):
            good_tw = np.flatnonzero(~np.isnan(msi_pop_geom_null[subj_i, n_i, fac_i, :]))
            rho, pv = stats.pearsonr(msi_pop_geom_null[subj_i, n_i, fac_i, good_tw], fac_coef[fac_i, good_tw])
            if np.isnan(rho):
                continue
            msi_coef_geom_null[subj_i, n_i, fac_i] = rho

        
# Compute Edge Null MSI x Expression Coefs
msi_coef_edge_null = np.zeros((n_subj, n_null, n_fac))
for subj_i, subj in enumerate(subj_dict.iterkeys()):
    for n_i, null_path in enumerate(subj_dict[subj]['subgraph_edge_null_path']):
        
        # Load Subject Data
        data_null_sub = np.load(null_path, mmap_mode='r')
        fac_coef = data_null_sub['fac_coef'] / data_null_sub['fac_coef'].max()
    
        # Get data params
        n_fac = fac_coef.shape[0]
        for fac_i in xrange(n_fac):
            good_tw = np.flatnonzero(~np.isnan(msi_pop_edge_null[subj_i, n_i, fac_i, :]))
            rho, pv = stats.pearsonr(msi_pop_edge_null[subj_i, n_i, fac_i, good_tw], fac_coef[fac_i, good_tw])
            if np.isnan(rho):
                continue
            msi_coef_edge_null[subj_i, n_i, fac_i] = rho


np.savez('{}/MSI_Coefs.npz'.format(path_ExpData),
         msi_coef=msi_coef,
         msi_coef_geom_null=msi_coef_geom_null,
         msi_coef_edge_null=msi_coef_edge_null)

### Plot Correlation between MSI and Subgraph Expression

In [ ]:
df = np.load('{}/MSI_Coefs.npz'.format(path_ExpData))
msi_coef = df['msi_coef']
msi_coef_geom_null = df['msi_coef_geom_null']
msi_coef_edge_null = df['msi_coef_edge_null']

# Get data params
n_subj, n_null, n_fac = msi_coef_geom_null.shape

# Sort the topographic sensitivity for subgraphs of each subject
msi_coef_sorted = np.sort(msi_coef, axis=1)
msi_coef_geom_null_sorted = np.sort(np.nanmean(msi_coef_geom_null, axis=1))
msi_coef_edge_null_sorted = np.sort(np.nanmean(msi_coef_edge_null, axis=1))

alpha = 0.05 / n_fac

print('\n\n----- Geometric Null ------')
for ii in xrange(n_fac):
    tval, pval = stats.ttest_rel(msi_coef_sorted[:, ii], msi_coef_geom_null_sorted[:, ii])
    if pval < alpha:
        sig = '*'
    else:
        sig = ' '    
    print('{}- t={:0.5}, p={:0.5}, sig={}'.format(ii+1, tval, pval, sig))


print('\n\n----- Edge Null ------')
for ii in xrange(n_fac):
    tval, pval = stats.ttest_rel(msi_coef_sorted[:, ii], msi_coef_edge_null_sorted[:, ii])
    if pval < alpha:
        sig = '*'
    else:
        sig = ' '    
    print('{}- t={:0.5}, p={:0.5}, sig={}'.format(ii+1, tval, pval, sig))
    
    
# Plot Module Sensitivity
plt.figure()
ax = plt.subplot(111)

ax.plot(np.linspace(1-0.25, n_fac+0.25, n_fac),
        np.percentile(msi_coef_geom_null_sorted, 50, axis=0),
        color='k', linewidth=0.5)
ax.fill_between([1-0.25, n_fac+0.25],
                y1=np.percentile(msi_coef_geom_null_sorted, 5),
                y2=np.percentile(msi_coef_geom_null_sorted, 95),
                color=[1.00, 0.25, 0.25], alpha=0.5, lw=0)


ax.plot(np.linspace(1-0.25, n_fac+0.25, n_fac),
        np.percentile(msi_coef_edge_null_sorted, 50, axis=0),
        color='k', linewidth=0.5)
ax.fill_between([1-0.25, n_fac+0.25],
                y1=np.percentile(msi_coef_edge_null_sorted, 5),
                y2=np.percentile(msi_coef_edge_null_sorted, 95),
                color=[0.75, 0.75, 0.75], alpha=0.5, lw=0)

clr_list = []
for ii in xrange(n_fac):
    clr_list.append([0.0, 0.375, 1.0])
bplot = ax.boxplot(msi_coef_sorted, patch_artist=True);
Echobase.Plotting.fig_format.set_box_color(bplot, 'k', clr_list)

# Axis Settings
ax.set_xlim([0, 17])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Module Sensitivity vs Expression');
ax.set_xlabel('Subgraphs');

plt.savefig('./e04a-Figures/MSI_Coef.svg')
plt.show()
plt.close()              

### Plot temporal expression

In [ ]:
df = np.load('{}/Module_Sensitivity_Index.npz'.format(path_ExpData))
msi_pop = df['msi_pop']

s_id = 1
msi_subj = msi_pop[s_id, :, :]
opt_fac = np.argsort(np.mean(msi_subj, axis=1))[-1]

# Load Subject Data
data_sub = np.load(subj_dict[subj_dict.keys()[s_id]]['subgraph_path'], mmap_mode='r')
fac_coef = data_sub['fac_coef'] / data_sub['fac_coef'].max()

### Plot Example Good and Bad Module sensitivity
plt.figure()
ax = plt.subplot(111)
ax.plot(fac_coef[opt_fac, :] / fac_coef[opt_fac, :].max())

# Axis Settings
ax.xaxis.set_ticks_position('bottom')
ax.set_xlabel('Time Windows');
ax.set_ylabel('Expression Coefficient');

ax1 = ax.twinx()
ax1.plot(msi_subj[opt_fac,:])

# Axis Settings
ax1.yaxis.set_ticks_position('right')
ax1.set_yticks([0.0, 0.2, 0.4, 0.6, 0.8, 1.0])
ax1.set_ylabel('Module Sensitivity');

plt.savefig('./e04a-Figures/{}-Subgraph_{}-Module_Sensitivity_Index-TemporalCoef.svg'.format(subj_dict.keys()[s_id], opt_fac))
plt.show()
plt.close()              